In [ ]:
pip install sentence-transformers tqdm


In [2]:
import pandas as pd
from nltk.tokenize import word_tokenize
from gensim.models import Doc2Vec
from tqdm import tqdm
from gensim.models.doc2vec import TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import ast

from sentence_transformers import SentenceTransformer

In [ ]:
import pandas as pd
import ast

# Carica i dataset
df_movies = pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv', low_memory=False)
df_keywords = pd.read_csv('/kaggle/input/the-movies-dataset/keywords.csv')
df_credits = pd.read_csv('/kaggle/input/the-movies-dataset/credits.csv')

# Converte gli id in stringa per tutti i DataFrame
df_movies['id'] = df_movies['id'].astype(str)
df_keywords['id'] = df_keywords['id'].astype(str)
df_credits['id'] = df_credits['id'].astype(str)

# Rimuove le righe con NaN nelle colonne 'original_title', 'title' e 'overview'
df_movies.dropna(subset=['original_title', 'title', 'overview'], inplace=True)

# Funzione per estrarre i nomi dei generi
def estrai_generi(genres_str):
    try:
        genres = ast.literal_eval(genres_str)
        return ', '.join([genre['name'] for genre in genres])
    except (ValueError, SyntaxError, TypeError):
        return ''

df_movies['genres_nomi'] = df_movies['genres'].apply(estrai_generi)

# Funzione per estrarre il regista (director)
def estrai_regista(crew_str):
    try:
        crew = ast.literal_eval(crew_str)
        for person in crew:
            if person.get('job') == 'Director':
                return person.get('name')
    except:
        return ''
    return ''

# Funzione per estrarre i primi 3 membri del cast
def estrai_cast(cast_str):
    try:
        cast = ast.literal_eval(cast_str)
        return ', '.join([actor['name'] for actor in cast[:3]])  # primi 3 attori
    except:
        return ''

# Estrai regista e cast
df_credits['regista'] = df_credits['crew'].apply(estrai_regista)
df_credits['cast_principale'] = df_credits['cast'].apply(estrai_cast)

# Unisci tutti i DataFrame
df_completo = df_movies.merge(df_keywords[['id', 'keywords']], on='id', how='left') \
                       .merge(df_credits[['id', 'regista', 'cast_principale']], on='id', how='left')

# Crea la colonna con le info combinate
df_completo['info_combinate'] = (
    'Overview: ' + df_completo['overview'].fillna('') + '\n' +
    'Generi: ' + df_completo['genres_nomi'] + '\n' +
    'Lingua originale: ' + df_completo['original_language'].fillna('') + '\n' +
    'Titolo originale: ' + df_completo['original_title'].fillna('') + '\n' +
    'Regista: ' + df_completo['regista'].fillna('') + '\n' +
    'Cast principale: ' + df_completo['cast_principale'].fillna('')
)

# Aggiungi nota se il film è per adulti
df_completo['info_combinate'] = df_completo.apply(
    lambda row: row['info_combinate'] + '\nThe Film is for adult' if str(row['adult']).lower() == 'true' else row['info_combinate'],
    axis=1
)

# Funzione per estrarre le parole chiave
def estrai_keywords(keywords_str):
    try:
        keywords = ast.literal_eval(keywords_str)
        return ', '.join([k['name'] for k in keywords])
    except:
        return ''

df_completo['keywords_str'] = df_completo['keywords'].apply(estrai_keywords)

# Aggiungi parole chiave (se presenti)
df_completo['info_combinate'] = df_completo.apply(
    lambda row: row['info_combinate'] + '\nKeywords: ' + row['keywords_str'] if row['keywords_str'] else row['info_combinate'],
    axis=1
)

# Crea il nuovo DataFrame finale
df_nuovo = df_completo[['id', 'info_combinate']].copy()

# Stampa il primo elemento
print(df_nuovo.iloc[0]['info_combinate'])
print(df_nuovo)


In [5]:
def ApplyBertEmbedding(Data, Feature, model_name='all-MiniLM-L6-v2', batch_size=8, output_file='bert_output.csv'):
    # Carica il modello SBERT pre-addestrato
    model = SentenceTransformer(model_name)

    # Preprocess: rimuovi NaN e sostituisci newline
    Data = Data.copy()
    Data = Data[Data[Feature].notna()]
    Data['Index'] = list(Data.index)
    Data['CleanedText'] = Data[Feature].str.replace('\n\n', ' ', regex=False)

    # Calcola gli embedding in batch
    all_embeddings = []
    for i in tqdm(range(0, len(Data), batch_size)):
        batch_texts = Data['CleanedText'].iloc[i:i+batch_size].tolist()
        batch_embeddings = model.encode(batch_texts, show_progress_bar=False)
        all_embeddings.extend(batch_embeddings)

    # Crea un DataFrame con gli embedding
    embeddings_df = pd.DataFrame(all_embeddings)
    embeddings_df['id'] = Data['id'].values

    # Salva su file
    embeddings_df.to_csv(output_file, index=False)

    return embeddings_df, model


In [ ]:
result_df, model = ApplyBertEmbedding(df_nuovo, 'info_combinate', batch_size=8, output_file='doc2vec_output.csv')

In [7]:
np.save('/kaggle/working/embeddings.npy', result_df.to_numpy())

In [8]:
plot = 'A adventure movies in a jungle'
#overview = jumanji['overview'].values[0].replace("\n\n", ' ')
overview = plot.replace("\n\n", ' ')

embedding = model.encode(overview)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
vectors_df = result_df.iloc[:, :-1].values  # I primi 300 sono i vettori, quindi prendi tutte le righe e le prime 300 colonne

# Ottieni il vettore di embedding del nuovo testo
# 'embedding' è il vettore del nuovo documento inferito
embedding = embedding.reshape(1, -1)  # Assicurati che il vettore sia una matrice di dimensione (1, 300)

# Calcola la similarità coseno tra l'embedding e tutti i vettori nel DataFrame
similarities = cosine_similarity(embedding, vectors_df)

# Ora, 'similarities' è un array che contiene le similarità coseno tra il nuovo vettore e tutti i vettori nel DataFrame
# Ordina i risultati per somiglianza decrescente e prendi i primi 5
top_5_indices = np.argsort(similarities[0])[::-1][:5]  # Ordina in ordine decrescente

# Estrai i titoli corrispondenti agli indici trovati
top_5_titles = result_df.iloc[top_5_indices, -1].values  # Assumiamo che la penultima colonna contenga i titoli

# Stampa i titoli dei 5 documenti più simili
print(top_5_titles)

In [ ]:
for idx in top_5_titles:
    plot = df_completo[df_completo['id'] == idx]['info_combinate'].iloc[0]
    print(f"{plot}\n")